In [5]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

In [24]:
train_files

array(['dogImages/train\\095.Kuvasz\\Kuvasz_06442.jpg',
       'dogImages/train\\057.Dalmatian\\Dalmatian_04054.jpg',
       'dogImages/train\\088.Irish_water_spaniel\\Irish_water_spaniel_06014.jpg',
       ..., 'dogImages/train\\029.Border_collie\\Border_collie_02069.jpg',
       'dogImages/train\\046.Cavalier_king_charles_spaniel\\Cavalier_king_charles_spaniel_03261.jpg',
       'dogImages/train\\048.Chihuahua\\Chihuahua_03416.jpg'],
      dtype='<U99')

In [25]:
train_targets

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [7]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [01:01<00:00, 13.58it/s]


In [29]:
train_tensors[1]

array([[[0.6313726 , 0.38431373, 0.01960784],
        [0.63529414, 0.3882353 , 0.02352941],
        [0.6313726 , 0.38431373, 0.01176471],
        ...,
        [0.4745098 , 0.08627451, 0.00392157],
        [0.44705883, 0.08627451, 0.        ],
        [0.47058824, 0.10980392, 0.01960784]],

       [[0.63529414, 0.3882353 , 0.02352941],
        [0.6313726 , 0.38431373, 0.01960784],
        [0.6392157 , 0.39215687, 0.01960784],
        ...,
        [0.4862745 , 0.09019608, 0.01176471],
        [0.4509804 , 0.08235294, 0.        ],
        [0.4745098 , 0.10588235, 0.00784314]],

       [[0.63529414, 0.3882353 , 0.02352941],
        [0.627451  , 0.38039216, 0.01568628],
        [0.6431373 , 0.39607844, 0.02352941],
        ...,
        [0.4862745 , 0.09019608, 0.01176471],
        [0.44313726, 0.06666667, 0.        ],
        [0.4627451 , 0.08627451, 0.        ]],

       ...,

       [[0.21176471, 0.10980392, 0.05098039],
        [0.21568628, 0.11372549, 0.05490196],
        [0.19607843, 0

In [8]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D,Dropout
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 



In [17]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 64)       

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

Epoch 1/20


184/334 [===============>..............] - ETA: 0s - loss: 4.9214 - accuracy: 0.0000e+ - ETA: 5:07 - loss: 11.1449 - accuracy: 0.0000e+0 - ETA: 7:45 - loss: 10.0211 - accuracy: 0.0000e+0 - ETA: 8:05 - loss: 8.9283 - accuracy: 0.0000e+0 - ETA: 8:04 - loss: 8.2185 - accuracy: 0.0000e+ - ETA: 8:09 - loss: 7.7070 - accuracy: 0.0000e+ - ETA: 8:38 - loss: 7.3409 - accuracy: 0.0000e+ - ETA: 8:36 - loss: 7.1026 - accuracy: 0.0000e+ - ETA: 8:39 - loss: 6.8593 - accuracy: 0.0000e+ - ETA: 8:51 - loss: 6.6884 - accuracy: 0.0000e+ - ETA: 8:55 - loss: 6.5306 - accuracy: 0.0045   - ETA: 8:56 - loss: 6.3968 - accuracy: 0.00 - ETA: 9:21 - loss: 6.2946 - accuracy: 0.00 - ETA: 9:16 - loss: 6.2047 - accuracy: 0.00 - ETA: 9:13 - loss: 6.1257 - accuracy: 0.00 - ETA: 9:02 - loss: 6.0565 - accuracy: 0.00 - ETA: 8:55 - loss: 5.9906 - accuracy: 0.00 - ETA: 8:47 - loss: 5.9339 - accuracy: 0.00 - ETA: 8:41 - loss: 5.8793 - accuracy: 0.00 - ETA: 8:32 - loss: 5.8307 - accuracy: 0.00 - ETA: 8:26 - loss: 5.7851 - acc

334/334 [==============================] - ETA: 3:36 - loss: 4.9765 - accuracy: 0.01 - ETA: 3:34 - loss: 4.9757 - accuracy: 0.01 - ETA: 3:33 - loss: 4.9747 - accuracy: 0.01 - ETA: 3:31 - loss: 4.9744 - accuracy: 0.01 - ETA: 3:30 - loss: 4.9737 - accuracy: 0.01 - ETA: 3:29 - loss: 4.9727 - accuracy: 0.01 - ETA: 3:27 - loss: 4.9715 - accuracy: 0.01 - ETA: 3:26 - loss: 4.9704 - accuracy: 0.01 - ETA: 3:24 - loss: 4.9689 - accuracy: 0.01 - ETA: 3:23 - loss: 4.9670 - accuracy: 0.01 - ETA: 3:21 - loss: 4.9654 - accuracy: 0.01 - ETA: 3:20 - loss: 4.9647 - accuracy: 0.01 - ETA: 3:18 - loss: 4.9640 - accuracy: 0.01 - ETA: 3:17 - loss: 4.9623 - accuracy: 0.01 - ETA: 3:15 - loss: 4.9603 - accuracy: 0.01 - ETA: 3:14 - loss: 4.9589 - accuracy: 0.01 - ETA: 3:12 - loss: 4.9579 - accuracy: 0.01 - ETA: 3:10 - loss: 4.9566 - accuracy: 0.01 - ETA: 3:09 - loss: 4.9561 - accuracy: 0.01 - ETA: 3:08 - loss: 4.9549 - accuracy: 0.01 - ETA: 3:06 - loss: 4.9533 - accuracy: 0.01 - ETA: 3:05 - loss: 4.9511 - accura

186/334 [===============>..............] - ETA: 0s - loss: 4.2558 - accuracy: 0.10 - ETA: 2:26 - loss: 4.3834 - accuracy: 0.05 - ETA: 3:14 - loss: 4.3846 - accuracy: 0.05 - ETA: 3:33 - loss: 4.4224 - accuracy: 0.03 - ETA: 3:46 - loss: 4.3398 - accuracy: 0.03 - ETA: 3:55 - loss: 4.3823 - accuracy: 0.03 - ETA: 4:11 - loss: 4.3461 - accuracy: 0.05 - ETA: 4:19 - loss: 4.4043 - accuracy: 0.05 - ETA: 4:23 - loss: 4.4170 - accuracy: 0.04 - ETA: 4:23 - loss: 4.4195 - accuracy: 0.04 - ETA: 4:23 - loss: 4.4236 - accuracy: 0.04 - ETA: 4:21 - loss: 4.4191 - accuracy: 0.03 - ETA: 4:19 - loss: 4.4208 - accuracy: 0.03 - ETA: 4:22 - loss: 4.4143 - accuracy: 0.03 - ETA: 4:25 - loss: 4.4148 - accuracy: 0.04 - ETA: 4:29 - loss: 4.4211 - accuracy: 0.04 - ETA: 4:28 - loss: 4.4276 - accuracy: 0.04 - ETA: 4:32 - loss: 4.4301 - accuracy: 0.04 - ETA: 4:33 - loss: 4.4331 - accuracy: 0.04 - ETA: 4:36 - loss: 4.4378 - accuracy: 0.04 - ETA: 4:38 - loss: 4.4408 - accuracy: 0.04 - ETA: 4:37 - loss: 4.4385 - accuracy

334/334 [==============================] - ETA: 2:06 - loss: 4.3803 - accuracy: 0.05 - ETA: 2:06 - loss: 4.3799 - accuracy: 0.05 - ETA: 2:05 - loss: 4.3791 - accuracy: 0.05 - ETA: 2:04 - loss: 4.3804 - accuracy: 0.05 - ETA: 2:03 - loss: 4.3801 - accuracy: 0.05 - ETA: 2:02 - loss: 4.3799 - accuracy: 0.05 - ETA: 2:02 - loss: 4.3790 - accuracy: 0.05 - ETA: 2:02 - loss: 4.3804 - accuracy: 0.05 - ETA: 2:01 - loss: 4.3790 - accuracy: 0.05 - ETA: 2:00 - loss: 4.3788 - accuracy: 0.05 - ETA: 1:59 - loss: 4.3775 - accuracy: 0.05 - ETA: 1:58 - loss: 4.3769 - accuracy: 0.05 - ETA: 1:58 - loss: 4.3793 - accuracy: 0.05 - ETA: 1:57 - loss: 4.3805 - accuracy: 0.05 - ETA: 1:56 - loss: 4.3804 - accuracy: 0.05 - ETA: 1:55 - loss: 4.3815 - accuracy: 0.05 - ETA: 1:54 - loss: 4.3810 - accuracy: 0.05 - ETA: 1:54 - loss: 4.3803 - accuracy: 0.05 - ETA: 1:53 - loss: 4.3793 - accuracy: 0.05 - ETA: 1:52 - loss: 4.3797 - accuracy: 0.05 - ETA: 1:51 - loss: 4.3800 - accuracy: 0.05 - ETA: 1:50 - loss: 4.3797 - accura

186/334 [===============>..............] - ETA: 0s - loss: 3.8881 - accuracy: 0.15 - ETA: 2:12 - loss: 3.8268 - accuracy: 0.12 - ETA: 2:57 - loss: 3.8883 - accuracy: 0.13 - ETA: 3:16 - loss: 3.8616 - accuracy: 0.15 - ETA: 3:33 - loss: 3.8526 - accuracy: 0.14 - ETA: 3:45 - loss: 3.8638 - accuracy: 0.15 - ETA: 3:51 - loss: 3.9017 - accuracy: 0.14 - ETA: 3:54 - loss: 3.9136 - accuracy: 0.13 - ETA: 3:55 - loss: 3.9041 - accuracy: 0.12 - ETA: 3:56 - loss: 3.9080 - accuracy: 0.12 - ETA: 3:56 - loss: 3.9014 - accuracy: 0.11 - ETA: 3:56 - loss: 3.8862 - accuracy: 0.12 - ETA: 3:56 - loss: 3.8627 - accuracy: 0.13 - ETA: 3:58 - loss: 3.8635 - accuracy: 0.12 - ETA: 3:59 - loss: 3.8815 - accuracy: 0.12 - ETA: 3:58 - loss: 3.8671 - accuracy: 0.11 - ETA: 3:58 - loss: 3.8697 - accuracy: 0.12 - ETA: 3:58 - loss: 3.8639 - accuracy: 0.11 - ETA: 3:59 - loss: 3.8967 - accuracy: 0.11 - ETA: 4:01 - loss: 3.9045 - accuracy: 0.11 - ETA: 4:02 - loss: 3.9110 - accuracy: 0.11 - ETA: 4:03 - loss: 3.8892 - accuracy

309/334 [==========================>...] - ETA: 1:57 - loss: 3.8841 - accuracy: 0.12 - ETA: 1:57 - loss: 3.8847 - accuracy: 0.11 - ETA: 1:56 - loss: 3.8848 - accuracy: 0.11 - ETA: 1:55 - loss: 3.8859 - accuracy: 0.11 - ETA: 1:54 - loss: 3.8893 - accuracy: 0.11 - ETA: 1:54 - loss: 3.8908 - accuracy: 0.11 - ETA: 1:53 - loss: 3.8903 - accuracy: 0.11 - ETA: 1:52 - loss: 3.8900 - accuracy: 0.11 - ETA: 1:51 - loss: 3.8890 - accuracy: 0.11 - ETA: 1:50 - loss: 3.8877 - accuracy: 0.11 - ETA: 1:50 - loss: 3.8832 - accuracy: 0.11 - ETA: 1:49 - loss: 3.8848 - accuracy: 0.11 - ETA: 1:48 - loss: 3.8855 - accuracy: 0.11 - ETA: 1:47 - loss: 3.8827 - accuracy: 0.12 - ETA: 1:46 - loss: 3.8812 - accuracy: 0.12 - ETA: 1:46 - loss: 3.8798 - accuracy: 0.12 - ETA: 1:45 - loss: 3.8836 - accuracy: 0.12 - ETA: 1:44 - loss: 3.8852 - accuracy: 0.12 - ETA: 1:44 - loss: 3.8834 - accuracy: 0.12 - ETA: 1:43 - loss: 3.8847 - accuracy: 0.12 - ETA: 1:42 - loss: 3.8850 - accuracy: 0.12 - ETA: 1:41 - loss: 3.8841 - accura

KeyboardInterrupt: 

In [20]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [21]:
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 6.2201%


In [22]:
#Test accuracy: 6.2201%

In [10]:
# Obtain bottleneck features from another pre-trained CNN(Xception)

bottleneck_features = np.load('bottleneck_features/DogXceptionData.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']

In [32]:
train_Xception[1]

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0. 

In [11]:
Xception_model = Sequential()
Xception_model.add(GlobalAveragePooling2D(input_shape=train_Xception.shape[1:]))
Xception_model.add(Dense(500, activation='relu'))
Xception_model.add(Dropout(0.4))
Xception_model.add(Dense(133, activation='softmax'))

Xception_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               1024500   
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               66633     
Total params: 1,091,133
Trainable params: 1,091,133
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the model.
Xception_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [13]:
#Train the model.
from keras.callbacks import ModelCheckpoint 

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.hdf5', 
                               verbose=1, save_best_only=True)

Xception_model.fit(train_Xception, train_targets, 
          validation_data=(valid_Xception, valid_targets),
          epochs=10, batch_size=20, callbacks=[checkpointer], verbose=1)

Epoch 1/10
334/334 [==============================] - ETA: 0s - loss: 4.8967 - accuracy: 0.0000e+ - ETA: 20s - loss: 5.0299 - accuracy: 0.0250   - ETA: 15s - loss: 5.0546 - accuracy: 0.037 - ETA: 16s - loss: 5.0037 - accuracy: 0.060 - ETA: 14s - loss: 4.7685 - accuracy: 0.085 - ETA: 13s - loss: 4.7863 - accuracy: 0.100 - ETA: 12s - loss: 4.7155 - accuracy: 0.118 - ETA: 12s - loss: 4.6343 - accuracy: 0.134 - ETA: 11s - loss: 4.5878 - accuracy: 0.140 - ETA: 11s - loss: 4.4927 - accuracy: 0.150 - ETA: 11s - loss: 4.4058 - accuracy: 0.160 - ETA: 10s - loss: 4.2909 - accuracy: 0.181 - ETA: 10s - loss: 4.2251 - accuracy: 0.193 - ETA: 10s - loss: 4.1586 - accuracy: 0.204 - ETA: 10s - loss: 4.1012 - accuracy: 0.209 - ETA: 10s - loss: 4.0687 - accuracy: 0.213 - ETA: 9s - loss: 3.9863 - accuracy: 0.232 - ETA: 9s - loss: 3.9405 - accuracy: 0.23 - ETA: 9s - loss: 3.8689 - accuracy: 0.25 - ETA: 9s - loss: 3.8127 - accuracy: 0.26 - ETA: 9s - loss: 3.7402 - accuracy: 0.27 - ETA: 9s - loss: 3.7118 - a

333/334 [============================>.] - ETA: 0s - loss: 0.5220 - accuracy: 0.75 - ETA: 5s - loss: 0.6223 - accuracy: 0.78 - ETA: 7s - loss: 0.6615 - accuracy: 0.78 - ETA: 8s - loss: 0.7544 - accuracy: 0.74 - ETA: 9s - loss: 0.7555 - accuracy: 0.75 - ETA: 9s - loss: 0.7104 - accuracy: 0.76 - ETA: 9s - loss: 0.7507 - accuracy: 0.76 - ETA: 9s - loss: 0.7422 - accuracy: 0.78 - ETA: 9s - loss: 0.7160 - accuracy: 0.79 - ETA: 9s - loss: 0.7306 - accuracy: 0.79 - ETA: 9s - loss: 0.6821 - accuracy: 0.80 - ETA: 9s - loss: 0.7112 - accuracy: 0.80 - ETA: 9s - loss: 0.7445 - accuracy: 0.78 - ETA: 9s - loss: 0.7373 - accuracy: 0.78 - ETA: 9s - loss: 0.7331 - accuracy: 0.78 - ETA: 9s - loss: 0.7161 - accuracy: 0.78 - ETA: 9s - loss: 0.6939 - accuracy: 0.79 - ETA: 9s - loss: 0.7106 - accuracy: 0.78 - ETA: 9s - loss: 0.7195 - accuracy: 0.78 - ETA: 9s - loss: 0.7427 - accuracy: 0.77 - ETA: 9s - loss: 0.7268 - accuracy: 0.78 - ETA: 9s - loss: 0.7441 - accuracy: 0.77 - ETA: 9s - loss: 0.7429 - accuracy

334/334 [==============================] - ETA: 0s - loss: 0.5120 - accuracy: 0.85 - ETA: 5s - loss: 0.5044 - accuracy: 0.83 - ETA: 7s - loss: 0.4088 - accuracy: 0.87 - ETA: 8s - loss: 0.4521 - accuracy: 0.84 - ETA: 8s - loss: 0.4220 - accuracy: 0.85 - ETA: 8s - loss: 0.4364 - accuracy: 0.84 - ETA: 8s - loss: 0.4525 - accuracy: 0.84 - ETA: 8s - loss: 0.4296 - accuracy: 0.85 - ETA: 8s - loss: 0.4237 - accuracy: 0.85 - ETA: 8s - loss: 0.4339 - accuracy: 0.85 - ETA: 8s - loss: 0.4616 - accuracy: 0.85 - ETA: 8s - loss: 0.4555 - accuracy: 0.84 - ETA: 8s - loss: 0.4668 - accuracy: 0.84 - ETA: 8s - loss: 0.4434 - accuracy: 0.85 - ETA: 8s - loss: 0.4472 - accuracy: 0.85 - ETA: 8s - loss: 0.4397 - accuracy: 0.85 - ETA: 8s - loss: 0.4286 - accuracy: 0.85 - ETA: 8s - loss: 0.4296 - accuracy: 0.85 - ETA: 7s - loss: 0.4428 - accuracy: 0.85 - ETA: 7s - loss: 0.4451 - accuracy: 0.85 - ETA: 7s - loss: 0.4460 - accuracy: 0.85 - ETA: 7s - loss: 0.4588 - accuracy: 0.84 - ETA: 7s - loss: 0.4727 - accuracy

332/334 [============================>.] - ETA: 0s - loss: 0.1398 - accuracy: 0.95 - ETA: 5s - loss: 0.1621 - accuracy: 0.93 - ETA: 7s - loss: 0.2121 - accuracy: 0.92 - ETA: 8s - loss: 0.2771 - accuracy: 0.90 - ETA: 8s - loss: 0.2864 - accuracy: 0.90 - ETA: 8s - loss: 0.2931 - accuracy: 0.90 - ETA: 8s - loss: 0.2906 - accuracy: 0.90 - ETA: 8s - loss: 0.2820 - accuracy: 0.90 - ETA: 8s - loss: 0.2778 - accuracy: 0.91 - ETA: 8s - loss: 0.3018 - accuracy: 0.91 - ETA: 8s - loss: 0.3015 - accuracy: 0.90 - ETA: 8s - loss: 0.3362 - accuracy: 0.89 - ETA: 8s - loss: 0.3350 - accuracy: 0.89 - ETA: 8s - loss: 0.3297 - accuracy: 0.89 - ETA: 8s - loss: 0.3269 - accuracy: 0.89 - ETA: 8s - loss: 0.3709 - accuracy: 0.89 - ETA: 8s - loss: 0.3722 - accuracy: 0.88 - ETA: 8s - loss: 0.3668 - accuracy: 0.89 - ETA: 8s - loss: 0.3611 - accuracy: 0.89 - ETA: 8s - loss: 0.3548 - accuracy: 0.89 - ETA: 8s - loss: 0.3550 - accuracy: 0.89 - ETA: 8s - loss: 0.3574 - accuracy: 0.89 - ETA: 8s - loss: 0.3500 - accuracy

333/334 [============================>.] - ETA: 0s - loss: 0.3121 - accuracy: 0.85 - ETA: 5s - loss: 0.2163 - accuracy: 0.91 - ETA: 6s - loss: 0.2935 - accuracy: 0.91 - ETA: 7s - loss: 0.3761 - accuracy: 0.90 - ETA: 8s - loss: 0.4758 - accuracy: 0.88 - ETA: 8s - loss: 0.4538 - accuracy: 0.89 - ETA: 8s - loss: 0.4033 - accuracy: 0.90 - ETA: 8s - loss: 0.4072 - accuracy: 0.90 - ETA: 8s - loss: 0.3886 - accuracy: 0.89 - ETA: 8s - loss: 0.3895 - accuracy: 0.89 - ETA: 8s - loss: 0.3830 - accuracy: 0.89 - ETA: 8s - loss: 0.3899 - accuracy: 0.89 - ETA: 8s - loss: 0.3905 - accuracy: 0.88 - ETA: 8s - loss: 0.3809 - accuracy: 0.88 - ETA: 8s - loss: 0.3867 - accuracy: 0.88 - ETA: 8s - loss: 0.3882 - accuracy: 0.88 - ETA: 8s - loss: 0.3846 - accuracy: 0.88 - ETA: 8s - loss: 0.3808 - accuracy: 0.88 - ETA: 8s - loss: 0.3686 - accuracy: 0.89 - ETA: 8s - loss: 0.3641 - accuracy: 0.89 - ETA: 8s - loss: 0.3684 - accuracy: 0.89 - ETA: 8s - loss: 0.3745 - accuracy: 0.88 - ETA: 8s - loss: 0.3673 - accuracy

334/334 [==============================] - ETA: 0s - loss: 0.0325 - accuracy: 1.00 - ETA: 7s - loss: 0.1686 - accuracy: 0.98 - ETA: 8s - loss: 0.2354 - accuracy: 0.93 - ETA: 9s - loss: 0.2360 - accuracy: 0.91 - ETA: 9s - loss: 0.3027 - accuracy: 0.91 - ETA: 10s - loss: 0.2725 - accuracy: 0.922 - ETA: 10s - loss: 0.2463 - accuracy: 0.926 - ETA: 10s - loss: 0.2326 - accuracy: 0.926 - ETA: 10s - loss: 0.2745 - accuracy: 0.920 - ETA: 10s - loss: 0.2903 - accuracy: 0.918 - ETA: 10s - loss: 0.2746 - accuracy: 0.919 - ETA: 10s - loss: 0.2923 - accuracy: 0.913 - ETA: 10s - loss: 0.2820 - accuracy: 0.912 - ETA: 10s - loss: 0.2875 - accuracy: 0.909 - ETA: 10s - loss: 0.2806 - accuracy: 0.910 - ETA: 10s - loss: 0.2823 - accuracy: 0.908 - ETA: 10s - loss: 0.2880 - accuracy: 0.907 - ETA: 10s - loss: 0.2783 - accuracy: 0.910 - ETA: 10s - loss: 0.2664 - accuracy: 0.914 - ETA: 10s - loss: 0.2759 - accuracy: 0.912 - ETA: 10s - loss: 0.2682 - accuracy: 0.913 - ETA: 10s - loss: 0.2715 - accuracy: 0.912 -

333/334 [============================>.] - ETA: 0s - loss: 0.1106 - accuracy: 0.95 - ETA: 8s - loss: 0.1372 - accuracy: 0.92 - ETA: 9s - loss: 0.1288 - accuracy: 0.93 - ETA: 10s - loss: 0.2076 - accuracy: 0.941 - ETA: 11s - loss: 0.2578 - accuracy: 0.925 - ETA: 11s - loss: 0.2570 - accuracy: 0.925 - ETA: 11s - loss: 0.2550 - accuracy: 0.929 - ETA: 11s - loss: 0.2444 - accuracy: 0.930 - ETA: 11s - loss: 0.2735 - accuracy: 0.926 - ETA: 11s - loss: 0.2554 - accuracy: 0.932 - ETA: 11s - loss: 0.2350 - accuracy: 0.939 - ETA: 11s - loss: 0.2397 - accuracy: 0.933 - ETA: 11s - loss: 0.2526 - accuracy: 0.921 - ETA: 11s - loss: 0.2449 - accuracy: 0.922 - ETA: 11s - loss: 0.2438 - accuracy: 0.924 - ETA: 11s - loss: 0.2345 - accuracy: 0.927 - ETA: 11s - loss: 0.2356 - accuracy: 0.927 - ETA: 10s - loss: 0.2332 - accuracy: 0.928 - ETA: 10s - loss: 0.2399 - accuracy: 0.928 - ETA: 10s - loss: 0.2431 - accuracy: 0.928 - ETA: 10s - loss: 0.2371 - accuracy: 0.930 - ETA: 10s - loss: 0.2286 - accuracy: 0.9

333/334 [============================>.] - ETA: 0s - loss: 0.0986 - accuracy: 0.95 - ETA: 6s - loss: 0.3526 - accuracy: 0.88 - ETA: 8s - loss: 0.2430 - accuracy: 0.91 - ETA: 9s - loss: 0.2821 - accuracy: 0.90 - ETA: 9s - loss: 0.2670 - accuracy: 0.89 - ETA: 9s - loss: 0.2490 - accuracy: 0.90 - ETA: 9s - loss: 0.3295 - accuracy: 0.90 - ETA: 9s - loss: 0.3324 - accuracy: 0.89 - ETA: 9s - loss: 0.3369 - accuracy: 0.89 - ETA: 9s - loss: 0.3500 - accuracy: 0.88 - ETA: 9s - loss: 0.3518 - accuracy: 0.88 - ETA: 9s - loss: 0.3396 - accuracy: 0.88 - ETA: 9s - loss: 0.3311 - accuracy: 0.89 - ETA: 9s - loss: 0.3352 - accuracy: 0.89 - ETA: 9s - loss: 0.3511 - accuracy: 0.88 - ETA: 9s - loss: 0.3418 - accuracy: 0.89 - ETA: 9s - loss: 0.3286 - accuracy: 0.89 - ETA: 9s - loss: 0.3325 - accuracy: 0.89 - ETA: 9s - loss: 0.3285 - accuracy: 0.89 - ETA: 9s - loss: 0.3262 - accuracy: 0.89 - ETA: 9s - loss: 0.3286 - accuracy: 0.89 - ETA: 8s - loss: 0.3270 - accuracy: 0.89 - ETA: 8s - loss: 0.3276 - accuracy

333/334 [============================>.] - ETA: 0s - loss: 0.0733 - accuracy: 0.95 - ETA: 7s - loss: 0.2377 - accuracy: 0.91 - ETA: 7s - loss: 0.2069 - accuracy: 0.93 - ETA: 8s - loss: 0.2832 - accuracy: 0.91 - ETA: 8s - loss: 0.2666 - accuracy: 0.92 - ETA: 8s - loss: 0.2563 - accuracy: 0.92 - ETA: 8s - loss: 0.2387 - accuracy: 0.92 - ETA: 8s - loss: 0.2242 - accuracy: 0.93 - ETA: 8s - loss: 0.2291 - accuracy: 0.93 - ETA: 8s - loss: 0.2063 - accuracy: 0.93 - ETA: 8s - loss: 0.2684 - accuracy: 0.92 - ETA: 8s - loss: 0.2525 - accuracy: 0.93 - ETA: 8s - loss: 0.2414 - accuracy: 0.93 - ETA: 8s - loss: 0.2354 - accuracy: 0.93 - ETA: 8s - loss: 0.2314 - accuracy: 0.93 - ETA: 8s - loss: 0.2239 - accuracy: 0.93 - ETA: 8s - loss: 0.2163 - accuracy: 0.93 - ETA: 8s - loss: 0.2077 - accuracy: 0.93 - ETA: 8s - loss: 0.2043 - accuracy: 0.93 - ETA: 8s - loss: 0.2025 - accuracy: 0.93 - ETA: 8s - loss: 0.2063 - accuracy: 0.93 - ETA: 8s - loss: 0.2102 - accuracy: 0.93 - ETA: 8s - loss: 0.2218 - accuracy

334/334 [==============================] - ETA: 0s - loss: 0.0614 - accuracy: 0.95 - ETA: 8s - loss: 0.0797 - accuracy: 0.95 - ETA: 9s - loss: 0.1268 - accuracy: 0.95 - ETA: 11s - loss: 0.1117 - accuracy: 0.960 - ETA: 10s - loss: 0.1068 - accuracy: 0.957 - ETA: 10s - loss: 0.1404 - accuracy: 0.955 - ETA: 10s - loss: 0.1445 - accuracy: 0.954 - ETA: 10s - loss: 0.1262 - accuracy: 0.961 - ETA: 10s - loss: 0.1117 - accuracy: 0.966 - ETA: 10s - loss: 0.1075 - accuracy: 0.967 - ETA: 9s - loss: 0.1289 - accuracy: 0.963 - ETA: 9s - loss: 0.1477 - accuracy: 0.95 - ETA: 9s - loss: 0.1571 - accuracy: 0.95 - ETA: 9s - loss: 0.1628 - accuracy: 0.95 - ETA: 9s - loss: 0.1675 - accuracy: 0.95 - ETA: 9s - loss: 0.1617 - accuracy: 0.95 - ETA: 9s - loss: 0.1631 - accuracy: 0.95 - ETA: 9s - loss: 0.1595 - accuracy: 0.95 - ETA: 9s - loss: 0.1619 - accuracy: 0.95 - ETA: 9s - loss: 0.1745 - accuracy: 0.94 - ETA: 9s - loss: 0.1873 - accuracy: 0.94 - ETA: 8s - loss: 0.1914 - accuracy: 0.93 - ETA: 8s - loss: 0.

In [14]:
#Load the model weights with the best validation loss.
Xception_model.load_weights('saved_models/weights.best.Xception.hdf5')

In [16]:
#Calculate classification accuracy on the test dataset.
Xception_predictions = [np.argmax(Xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Xception]
test_accuracy = 100*np.mean(np.array(Xception_predictions)==np.argmax(test_targets, axis=1))
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.6986%


In [23]:
# Test accuracy: 81.6986%